In [42]:
import mysql.connector as msql
from mysql.connector import Error
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [43]:
mydb = msql.connect(
  host="localhost",
  user="root",
  password="root",
  database="trajectories"
)
mycursor = mydb.cursor(buffered=True)

try:
    if mydb.is_connected():

        query = """SELECT * FROM trajectories.features;"""
        mycursor.execute(query)
        data = mycursor.fetchall()
        mydb.commit()

except Error as e:

    print("Error while connecting to gpspoints:", e)

finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection is closed")

MySQL connection is closed


In [44]:
# df = pd.DataFrame(data, columns = ['max_dist', 'avg_dist', 'min_dist', 'var_dist', 'shannon_dist', 'fisher_dist', 'pearson_dist', 
#                                   'max_speed', 'avg_speed', 'min_speed', 'var_speed', 'shannon_speed', 'fisher_speed', 'pearson_speed',
#                                  'max_acc', 'avg_acc', 'min_acc', 'var_acc', 'shannon_acc', 'fisher_acc', 'pearson_acc', 'label'])
# X = df[['max_dist', 'avg_dist', 'min_dist', 'var_dist', 'shannon_dist', 'fisher_dist', 'pearson_dist', 
#                                   'max_speed', 'avg_speed', 'min_speed', 'var_speed', 'shannon_speed', 'fisher_speed', 'pearson_speed',
#                                  'max_acc', 'avg_acc', 'min_acc', 'var_acc', 'shannon_acc', 'fisher_acc', 'pearson_acc']]

df = pd.DataFrame(data, columns = ['max_duration', 'avg_duration', 'min_duration', 'shannon_duration', 'fisher_duration',
                                   'pearson_duration', 'max_length', 'avg_length', 'min_length', 'shannon_length', 
                                   'fisher_length', 'pearson_length', 'label'])
X = df[['max_duration', 'avg_duration', 'min_duration', 'shannon_duration', 'fisher_duration', 'pearson_duration', 
        'max_length', 'avg_length', 'min_length', 'shannon_length', 'fisher_length', 'pearson_length']]
y = df[['label']].values.ravel()
print(X.shape)

(2106, 21)


In [56]:
def random_forest_model(X, y):
    n_features_to_select = [.2, .5, .7, 1.0]
    for n in n_features_to_select:
        print(n,"Random Forest Model features ----------------------------")
        clf = RandomForestClassifier()
        selector = RFE(clf, n_features_to_select = n)
        features = selector.fit(X, y).get_feature_names_out()
        print(features)

        X_new = df[features]
        X_train, X_test, y_train, y_test = train_test_split( X_new, y, test_size=0.2, random_state=42)

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        score = f1_score(y_test, y_pred, average='micro')
        print(score)
    
def adaboost_model(X, y):
    n_features_to_select = [.2, .5, .7, 1.0]
    for n in n_features_to_select:
        print(n,"AdaBoost Model features -------------------------------------")
        clf = AdaBoostClassifier()
        selector = RFE(clf, n_features_to_select = n)
        features = selector.fit(X, y).get_feature_names_out()
        print(features)

        X_new = df[features]
        X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        score = f1_score(y_test, y_pred, average='micro')
        print(score)

In [57]:
X, y = train_test_data(data)
random_forest_model(X, y)

(2106, 21)
0.2 Random Forest Model features ----------------------------
['avg_dist' 'avg_speed' 'avg_acc' 'shannon_acc']
0.7654028436018957
0.5 Random Forest Model features ----------------------------
['avg_dist' 'var_dist' 'shannon_dist' 'avg_speed' 'shannon_speed'
 'fisher_speed' 'max_acc' 'avg_acc' 'var_acc' 'shannon_acc']
0.8009478672985783
0.7 Random Forest Model features ----------------------------
['avg_dist' 'var_dist' 'shannon_dist' 'pearson_dist' 'max_speed'
 'avg_speed' 'var_speed' 'shannon_speed' 'fisher_speed' 'pearson_speed'
 'max_acc' 'avg_acc' 'var_acc' 'shannon_acc']
0.8222748815165877
1.0 Random Forest Model features ----------------------------
['max_dist' 'avg_dist' 'min_dist' 'var_dist' 'shannon_dist' 'fisher_dist'
 'pearson_dist' 'max_speed' 'avg_speed' 'min_speed' 'var_speed'
 'shannon_speed' 'fisher_speed' 'pearson_speed' 'max_acc' 'avg_acc'
 'min_acc' 'var_acc' 'shannon_acc' 'fisher_acc' 'pearson_acc']
0.8317535545023697
